
# Tampa Data Meetup - Feb '23
#### Databricks & PowerBI w/ ChatGPT





<img src = 'https://raw.githubusercontent.com/Evogelpohl/linkArtifacts/main/mj02.jpg' >


<b>Midjourney</b>: prompt: group of 30 different cartoonish happy and impressed robots sitting at a presentation meeting room looking at the speaker on stage large room. facing the speaker. vfx. sharp focus --ar 16:9 --s 250

# WARNING: LLMs are not always right. 

<img src='https://raw.githubusercontent.com/Evogelpohl/linkArtifacts/main/oops_age.png' width=125% img>


# Data Challenge: Have ChatGPT Help Us Create a DW Entry for NY Taxi Rides & Analyze it


<img src='https://raw.githubusercontent.com/Evogelpohl/linkArtifacts/main/DLT_graphic_tiers.jpg'>



# SETUP: Use the Unity Catalog by Default

In [0]:
%sql

--Switch to our demo catalog
USE CATALOG demos;

--Switch to our demo schema/database
USE SCHEMA meetup;

INFO:py4j.clientserver:Received command c on object id p0



# DATA: CSVs for NYC Yellow Taxi Rides:

<img src='https://raw.githubusercontent.com/Evogelpohl/linkArtifacts/main/nytaxi_schema.png' img>


## Let's ask ChatGPT to create a python snippet to read those files into a dataframe. 
#### All we know is the path to our CSV on Azure Blog Storage (like S3):

In [0]:
nycTaxi_CSVs = 'abfss://data@bpdemostorage01.dfs.core.windows.net/raw_data/yellow_taxi_csv/'

display(dbutils.fs.ls(nycTaxi_CSVs))

INFO:py4j.clientserver:Received command c on object id p0


path,name,size,modificationTime
abfss://data@bpdemostorage01.dfs.core.windows.net/raw_data/yellow_taxi_csv/yellow_tripdata_2021-01.csv,yellow_tripdata_2021-01.csv,151915921,1677441664000
abfss://data@bpdemostorage01.dfs.core.windows.net/raw_data/yellow_taxi_csv/yellow_tripdata_2021-02.csv,yellow_tripdata_2021-02.csv,152161840,1677441682000
abfss://data@bpdemostorage01.dfs.core.windows.net/raw_data/yellow_taxi_csv/yellow_tripdata_2021-03.csv,yellow_tripdata_2021-03.csv,214246967,1677441702000
abfss://data@bpdemostorage01.dfs.core.windows.net/raw_data/yellow_taxi_csv/yellow_tripdata_2021-04.csv,yellow_tripdata_2021-04.csv,247567003,1677441723000
abfss://data@bpdemostorage01.dfs.core.windows.net/raw_data/yellow_taxi_csv/yellow_tripdata_2021-05.csv,yellow_tripdata_2021-05.csv,286350897,1677441745000
abfss://data@bpdemostorage01.dfs.core.windows.net/raw_data/yellow_taxi_csv/yellow_tripdata_2021-06.csv,yellow_tripdata_2021-06.csv,324145437,1677441772000
abfss://data@bpdemostorage01.dfs.core.windows.net/raw_data/yellow_taxi_csv/yellow_tripdata_2021-07.csv,yellow_tripdata_2021-07.csv,322643576,1677441801000
abfss://data@bpdemostorage01.dfs.core.windows.net/raw_data/yellow_taxi_csv/yellow_tripdata_2021-08.csv,yellow_tripdata_2021-08.csv,318860913,1677441825000
abfss://data@bpdemostorage01.dfs.core.windows.net/raw_data/yellow_taxi_csv/yellow_tripdata_2021-09.csv,yellow_tripdata_2021-09.csv,339035816,1677441850000
abfss://data@bpdemostorage01.dfs.core.windows.net/raw_data/yellow_taxi_csv/yellow_tripdata_2021-10.csv,yellow_tripdata_2021-10.csv,396643647,1677441878000



# CREATE: Bronze table from df


## Prompt: 

I'm using Databricks, in pySpark dialect. 

I have all NY City Yellow Taxi rides for the past few years as CSVs in a path on azure storage. The abfss:// path is in variable 'nycTaxi_CSVs'

Each CSV has a header. Create a dataframe for all the records please.


In [0]:
# Infer the schema from the first CSV file and create a DataFrame for it
sample_csv = nycTaxi_CSVs + "yellow_tripdata_2022-01.csv"
sample_df = spark.read.format("csv") \
                    .option("header", "true") \
                    .option("inferSchema", "true") \
                    .load(sample_csv)

# Create the final DataFrame for all CSV files, using the schema inferred from the sample DataFrame
df = spark.read.format("csv") \
              .option("header", "true") \
              .schema(sample_df.schema) \
              .load(nycTaxi_CSVs)


INFO:py4j.clientserver:Received command c on object id p0


In [0]:
# Drop the table if it exists.
sql('DROP TABLE IF EXISTS trips_bronze;')

# Create the bronze table
df.write.saveAsTable('trips_bronze')


INFO:py4j.clientserver:Received command c on object id p0



# CREATE: Silver. Read into df_cleansed


### Prompt

I've written the df to a delta table for speed. I now have a new dataframe, df_cleased for future processing.

In [0]:
display(df.printSchema())

INFO:py4j.clientserver:Received command c on object id p0
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [0]:
df_cleansed = spark.table('trips_bronze')
df_cleansed.printSchema()

INFO:py4j.clientserver:Received command c on object id p0
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)




## TRANSFORM: Clean the dataframe with business-friendly column names


### Prompt #3:

The schema for our df_cleansed dataframe is as follows. Can you please write the code to apply a new schema that is has business-friendly column names, with proper case and let's use underscores for spaces? We can keep the same df_cleansed name. 

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)


In [0]:
from pyspark.sql.functions import col

# create a dictionary of old and new column names
new_column_names = {
    "VendorID": "vendor_id",
    "tpep_pickup_datetime": "pickup_datetime",
    "tpep_dropoff_datetime": "dropoff_datetime",
    "passenger_count": "passenger_count",
    "trip_distance": "trip_distance",
    "RatecodeID": "rate_code_id",
    "store_and_fwd_flag": "store_and_fwd_flag",
    "PULocationID": "pickup_location_id",
    "DOLocationID": "dropoff_location_id",
    "payment_type": "payment_type",
    "fare_amount": "fare_amount",
    "extra": "extra",
    "mta_tax": "mta_tax",
    "tip_amount": "tip_amount",
    "tolls_amount": "tolls_amount",
    "improvement_surcharge": "improvement_surcharge",
    "total_amount": "total_amount",
    "congestion_surcharge": "congestion_surcharge",
    "airport_fee": "airport_fee"
}

# create a list of expressions for the new schema
new_schema_exprs = [col(c).alias(new_column_names.get(c, c)) for c in df_cleansed.columns]

# apply the new schema to the dataframe
df_cleansed = df_cleansed.select(new_schema_exprs)

# show the new schema
df_cleansed.printSchema()


INFO:py4j.clientserver:Received command c on object id p0
root
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- rate_code_id: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)




## TRANSFORM: Apply business rules to our data


### Prompt:

That schema is great. It worked. Next, I've done some analysis and determined that the following business rules we need to use to update the dataframe:

- when Vendor_id is 1, replace with 'CMTech', when it's 2, replace with 'Verifone'.
- when payment_type is 1, 'credit', when 2, cash, 3, No_Charge, 4, Dispute, 5, Unknown, 6, Voided_Trip
- when Rate_Code_id is 1, Standard_rate, 2, JFK, 3, Newark, 4, Nassau/Westchester, 5, Negotiated_Fare, 6, Group_Ride

all other columns can stay as they are. Can you write the code to update our df_cleansed dataframe?

In [0]:
from pyspark.sql.functions import when

# define the mapping for vendor_id
vendor_id_map = {
    1: "CMTech",
    2: "Verifone"
}

# define the mapping for payment_type
payment_type_map = {
    1: "credit",
    2: "cash",
    3: "No_Charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided_Trip"
}

# define the mapping for rate_code_id
rate_code_id_map = {
    1: "Standard_rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau/Westchester",
    5: "Negotiated_Fare",
    6: "Group_Ride"
}

# apply the mappings to the relevant columns
df_cleansed = df_cleansed.withColumn("vendor_id", when(col("vendor_id") == 1, "CMTech").otherwise("Verifone"))
df_cleansed = df_cleansed.withColumn("payment_type", when(col("payment_type") == 1, "credit")
                                     .when(col("payment_type") == 2, "cash")
                                     .when(col("payment_type") == 3, "No_Charge")
                                     .when(col("payment_type") == 4, "Dispute")
                                     .when(col("payment_type") == 5, "Unknown")
                                     .otherwise("Voided_Trip"))
df_cleansed = df_cleansed.withColumn("rate_code_id", when(col("rate_code_id") == 1, "Standard_rate")
                                     .when(col("rate_code_id") == 2, "JFK")
                                     .when(col("rate_code_id") == 3, "Newark")
                                     .when(col("rate_code_id") == 4, "Nassau/Westchester")
                                     .when(col("rate_code_id") == 5, "Negotiated_Fare")
                                     .otherwise("Group_Ride"))

# show the updated dataframe
display(df_cleansed)


INFO:py4j.clientserver:Received command c on object id p0


vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
CMTech,2021-01-01T00:30:10.000+0000,2021-01-01T00:36:12.000+0000,1.0,2.1,Standard_rate,N,142,43,cash,8.0,3.0,0.5,0.0,0.0,0.3,11.8,2.5,null
CMTech,2021-01-01T00:51:20.000+0000,2021-01-01T00:52:19.000+0000,1.0,0.2,Standard_rate,N,238,151,cash,3.0,0.5,0.5,0.0,0.0,0.3,4.3,0.0,null
CMTech,2021-01-01T00:43:30.000+0000,2021-01-01T01:11:06.000+0000,1.0,14.7,Standard_rate,N,132,165,credit,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,null
CMTech,2021-01-01T00:15:48.000+0000,2021-01-01T00:31:01.000+0000,0.0,10.6,Standard_rate,N,138,132,credit,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,null
Verifone,2021-01-01T00:31:49.000+0000,2021-01-01T00:48:21.000+0000,1.0,4.94,Standard_rate,N,68,33,credit,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,null
CMTech,2021-01-01T00:16:29.000+0000,2021-01-01T00:24:30.000+0000,1.0,1.6,Standard_rate,N,224,68,credit,8.0,3.0,0.5,2.35,0.0,0.3,14.15,2.5,null
CMTech,2021-01-01T00:00:28.000+0000,2021-01-01T00:17:28.000+0000,1.0,4.1,Standard_rate,N,95,157,cash,16.0,0.5,0.5,0.0,0.0,0.3,17.3,0.0,null
CMTech,2021-01-01T00:12:29.000+0000,2021-01-01T00:30:34.000+0000,1.0,5.7,Standard_rate,N,90,40,cash,18.0,3.0,0.5,0.0,0.0,0.3,21.8,2.5,null
CMTech,2021-01-01T00:39:16.000+0000,2021-01-01T01:00:13.000+0000,1.0,9.1,Standard_rate,N,97,129,Dispute,27.5,0.5,0.5,0.0,0.0,0.3,28.8,0.0,null
CMTech,2021-01-01T00:26:12.000+0000,2021-01-01T00:39:46.000+0000,2.0,2.7,Standard_rate,N,263,142,credit,12.0,3.0,0.5,3.15,0.0,0.3,18.95,2.5,null



<img src='https://raw.githubusercontent.com/Evogelpohl/linkArtifacts/main/thankyou.png' width=120%>



## TRANSFORM: Look for bad data

In [0]:
# This code drops an existing table named trips_silver, if it exists. It then saves the cleaned DataFrame as a new table named trips_silver, and optimizes it for better performance. Finally, it reloads the table as a new DataFrame.

# Drop existing table trips_silver, if it exists
sql('drop table if exists trips_silver;')

# Save cleaned DataFrame as new table trips_silver, and overwrite if it already exists
df_cleansed.write.mode('overwrite').saveAsTable('trips_silver')

# Optimize trips_silver table for better performance
sql('OPTIMIZE trips_silver')

# Reload trips_silver table as new DataFrame
df_cleansed = spark.table('trips_silver')


INFO:py4j.clientserver:Received command c on object id p0



### Prompt:

Thank you. Our df_cleansed dataframe has business rules applied now, but we might have bad data in our data set. Can you suggest ways of removing them? For example, pickup and dropoff dates should only be in the years 2021 and 2022.


In [0]:
from pyspark.sql.functions import *

# Filter out bad data by checking if Fare_Amount, Tip_Amount, Tolls_Amount, and Total_Amount are non-negative
df_cleansed = df_cleansed.filter((col("Fare_Amount") >= 0) &
                                 (col("Tip_Amount") >= 0) &
                                 (col("Tolls_Amount") >= 0) &
                                 (col("Total_Amount") >= 0))



# filter out records with invalid pickup and dropoff dates
df_cleansed = df_cleansed.filter(
    (year(col("pickup_datetime")).isin([2021, 2022])) & 
    (year(col("dropoff_datetime")).isin([2021, 2022]))
)


# Print the updated DataFrame
display(df_cleansed.count())

INFO:py4j.clientserver:Received command c on object id p0
66791349

In [0]:
df_cleansed.write.mode('overwrite').saveAsTable('trips_silver')

sql('OPTIMIZE trips_silver')

df_cleansed = spark.table('trips_silver')

df_cleansed.printSchema()

INFO:py4j.clientserver:Received command c on object id p0
root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- rate_code_id: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)




# ANALYZE: Like a boss!


<img src='https://raw.githubusercontent.com/Evogelpohl/linkArtifacts/main/impress_boss.jpg' width=120%>



#Prompt:

I wrote the df_cleansed dataframe to a delta table. So, I can switch to SQL and impress the boss with my analysis of the taxi data. Can you suggest a few insightful queries showing interesting trends or facts within the dataset? Please use SQL, not Python. My table is called trips_silver and has the following schema.

trips_silver schema: 

root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- rate_code_id: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)


In [0]:
%sql

SELECT vendor_id, SUM(total_amount) as total_revenue
FROM trips_silver
GROUP BY vendor_id


INFO:py4j.clientserver:Received command c on object id p0


vendor_id,total_revenue
CMTech,3.911058250543356E8
Verifone,9.997056489056872E8


Databricks visualization. Run in Databricks to view.


<img src='https://raw.githubusercontent.com/Evogelpohl/linkArtifacts/main/bam.png' width=120%>



### Prompt

Can we write a SQL query to show the average fare_amount per mile (trip_distance) for the top 2 pickup_location_ids for each month-year? I'll use this to build a visual to show a trend. 

And to be fair for the comparison, how should we normalize trip_distance for the two locations so we remove extreme outliers?

In [0]:
%sql 

WITH pickup_location_counts AS (
  SELECT pickup_location_id, COUNT(*) AS num_trips
  FROM trips_silver
  GROUP BY pickup_location_id
  ORDER BY num_trips DESC
  LIMIT 2
),


pickup_location_fares AS (
  SELECT 
    DATE_TRUNC('month', pickup_datetime) AS pickup_month,
    YEAR(pickup_datetime) AS pickup_year,
    pickup_location_id, 
    fare_amount / trip_distance AS fare_per_mile
  FROM trips_silver
  WHERE pickup_location_id IN (SELECT pickup_location_id FROM pickup_location_counts)
)


SELECT 
  CONCAT(pickup_year, '-', LPAD(MONTH(pickup_month), 2, '0')) AS pickup_month_year,
  pickup_location_id, 
  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY fare_per_mile) AS median_fare_per_mile
FROM (
  SELECT 
    pickup_month,
    pickup_year,
    pickup_location_id,
    fare_per_mile,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY fare_per_mile) OVER (PARTITION BY pickup_month, pickup_location_id) AS q3,
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY fare_per_mile) OVER (PARTITION BY pickup_month, pickup_location_id) AS q1
  FROM pickup_location_fares
) filtered_fares
WHERE fare_per_mile BETWEEN q1 - 1.5 * (q3 - q1) AND q3 + 1.5 * (q3 - q1)
GROUP BY pickup_year, MONTH(pickup_month), pickup_location_id
ORDER BY pickup_year, MONTH(pickup_month), pickup_location_id



INFO:py4j.clientserver:Received command c on object id p0


pickup_month_year,pickup_location_id,median_fare_per_mile
2021-01,236,5.357142857142857
2021-01,237,5.416666666666667
2021-02,236,5.520169851380042
2021-02,237,5.555555555555556
2021-03,236,5.416666666666667
2021-03,237,5.476190476190476
2021-04,236,5.466237942122187
2021-04,237,5.555555555555555
2021-05,236,5.483870967741935
2021-05,237,5.597014925373134


Databricks visualization. Run in Databricks to view.


## Prompt 

I might have embellished my resume a bit. Can you help me with some data science to impress my boss? 

Can you suggest a good Python module for forecasting data like this? Let's write the Python using df_cleansed and forecast trips for the next 6 months based on hourly data. And can you please document the code so I can learn?



In [0]:
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from fbprophet import Prophet
from pyspark.sql.functions import to_date, hour

# Convert the pickup_datetime column to a date column
df_cleansed = df_cleansed.withColumn('pickup_date', to_date('pickup_datetime'))

# Group data by hour and sum the total_amount column
df_hourly_grouped = df_cleansed.groupBy(date_trunc('hour', 'pickup_datetime').alias('pickup_hour')).sum('total_amount')

# Rename columns to match Prophet's requirements
df_hourly = df_hourly_grouped.selectExpr("pickup_hour as ds", "`sum(total_amount)` as y")

# Initialize Prophet model
model = Prophet()

# Fit the model to the data
model.fit(df_hourly.toPandas())

# Create dataframe with future dates
future = model.make_future_dataframe(periods=6*30*24, freq='H')

# Generate forecast for future dates
forecast_pd = model.predict(future)
forecast = spark.createDataFrame(forecast_pd)

# Print the forecast
forecast.select('ds', 'yhat', 'yhat_lower', 'yhat_upper').show()


INFO:py4j.clientserver:Received command c on object id p0
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
+-------------------+-------------------+-------------------+------------------+
|                 ds|               yhat|         yhat_lower|        yhat_upper|
+-------------------+-------------------+-------------------+------------------+
|2021-01-01 00:00:00|  11515.76976178146|-15631.767097621369| 39978.47861444474|
|2021-01-01 01:00:00|  -8220.43274077206| -35725.30516179413|19318.481888519367|
|2021-01-01 02:00:00|-25456.017467280515| -51615.49867366597|3377.9517471466947|
|2021-01-01 03:00:00|-34853.470663810935|  -62178.9439965652|-7548.905950735374|
|2021-01-01 04:00:00| -33642.67377284152| -64460.83254079037|-7536.766744399211|
|2021-01-01 05:00:00|-22923.346274936594|-50935.217327919156| 4661.352602620524|
|2021-01-01 06:00:00| -6582.408499413887| -35123.86389283607|22533.185901956447|
|2021-01-01 07:00:00| 11004.